<a href="https://colab.research.google.com/github/ApoorvaNagarajan/Image-Filter-Kernels/blob/master/assignment4/assignment4_iteration2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries and modules**

**Idea behind the network**

- Has three improvements over iteration1
- Improvements
  - Reduce number of channels : To bring down number of parameters. Reducing such that we dont break our basic architectural idea of gradually increasing number of channels at convolution layers and reducing channels at the transition layer
  - Increase batch size to 128 - To improve accuracy
  - Add batch normalization to avoid overflow after every convolution layer

**Observations : Filled after running the network below. (Have put it in the top so that this is not missed out)**

- Parameters : 11,250
- Epochs : 10
- val_acc : 0.9879 - This is lesser than the first iteration
- Time taken for each epoch : 120us
- First two val_acc : 0.9782, 0.9764 : This is lesser than the DNN in 1st iteration. This implies that the accuracy will never match that of the first network usless other parameters are tweaked
- With each iteration, training accuracy is increasing. But the testing accuracy is not. This means the network is over fitting
- Number of parameters are good enough. I need to improve the accuracy keeping overfitting in mind in the next iteration

In [0]:
# Import all the required python libraries
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Add, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

from keras.datasets import mnist

### Load pre-shuffled MNIST data into train and test sets

In [0]:
# downloads the MNIST dataset and splits it into test and train.
(X_train, y_train), (X_test, y_test) = mnist.load_data()


# X_train and X_test are reshaped from 3 dimensions to 4 dimensions
# 1st dim : num images
# 2nd and 3rd dim : Width and height of eaach of the images
# 4th dim : number of channels in each of the image. We set this to 1 as the
#           our dataset has only gray images
X_train = X_train.reshape(X_train.shape[0], 28, 28,1)
X_test = X_test.reshape(X_test.shape[0], 28, 28,1)


# Normalizing the image
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


# Convert 1-dimensional class arrays to 10-dimensional class matrices
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

In [13]:
from keras.layers import Activation
model = Sequential()

 
model.add(Convolution2D(10, 3, 3, activation='relu', input_shape=(28,28,1))) # receptive field: 3x3, output dim: 26x26x10
model.add(BatchNormalization())
model.add(Convolution2D(10, 3, 3, activation='relu')) # receptive field: 5x5, output dim: 24x24x10
model.add(BatchNormalization())
model.add(Convolution2D(20, 3, 3, activation='relu'))  # receptive field: 7x7, output dim: 22x22x20
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))  # receptive field: 14x14, output dim: 11x11x20
model.add(Convolution2D(10, 1, 1, activation='relu')) # receptive field: 14x14, output dim: 11x11x10
model.add(BatchNormalization())

model.add(Convolution2D(10, 3, 3, activation='relu')) # receptive field: 16x16, output dim: 9x9x10
model.add(BatchNormalization())
model.add(Convolution2D(20, 3, 3, activation='relu')) # receptive field: 18x18, output dim: 7x7x20
model.add(BatchNormalization())

model.add(Convolution2D(10, 1, activation='relu')) # receptive field: 18x18, output dim: 7x7x10
model.add(BatchNormalization())
model.add(Convolution2D(10, 7)) # receptive field: 25x25, output dim: 1x1x10
model.add(Flatten())
model.add(Activation('softmax'))


model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (3, 3), activation="relu", input_shape=(28, 28, 1...)`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (3, 3), activation="relu")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (3, 3), activation="relu")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (1, 1), activation="relu")`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (3, 3), activation="relu")`
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 26, 26, 10)        100       
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 26, 10)        40        
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 24, 24, 10)        910       
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 10)        40        
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 22, 22, 20)        1820      
_________________________________________________________________
batch_normalization_3 (Batch (None, 22, 22, 20)        80        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 11, 11, 20)        0         
__________

In [0]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [15]:
model.fit(X_train, Y_train, batch_size=128, nb_epoch=10, validation_data=(X_test, Y_test), verbose=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 122us/step - loss: 0.2390 - acc: 0.9272 - val_loss: 0.0696 - val_acc: 0.9782
Epoch 2/10
60000/60000 [==============================] - 6s 102us/step - loss: 0.0588 - acc: 0.9821 - val_loss: 0.0766 - val_acc: 0.9764
Epoch 3/10
60000/60000 [==============================] - 6s 102us/step - loss: 0.0417 - acc: 0.9873 - val_loss: 0.0420 - val_acc: 0.9863
Epoch 4/10
60000/60000 [==============================] - 6s 102us/step - loss: 0.0341 - acc: 0.9892 - val_loss: 0.0383 - val_acc: 0.9883
Epoch 5/10
60000/60000 [==============================] - 6s 102us/step - loss: 0.0292 - acc: 0.9907 - val_loss: 0.0337 - val_acc: 0.9895
Epoch 6/10
60000/60000 [==============================] - 6s 102us/step - loss: 0.0240 - acc: 0.9926 - val_loss: 0.0423 - val_acc: 0.9863
Epoch 7/10
60000/60000 [==============================] - 6s 102us/step - loss: 0.0226 - acc: 0.9926 - val_loss: 0.0343 - 

In [16]:
score = model.evaluate(X_test, Y_test, verbose=0)
print(score)

[0.03934841539740446, 0.9879]
